# Microbial Community Disease Risk Prediction

This case study involves developing and evaluating neural network models to predict disease risk from high-dimensional microbiome sequence data. Our dataset consists of 16S rRNA gene profiles from 16,344 samples, with approximately half of the samples coming from individuals diagnosed with type 1 diabetes (cases) and the other half from individuals without a diagnosis of type 1 diabetes (controls). By modeling patterns in these microbial community profiles, our goal is to accurately classify samples based on disease status. This could provide insight into whether an individual's gut microbiome composition is associated with their risk of developing type 1 diabetes. We implement and compare several neural network architectures, including convolutional and recurrent networks, to evaluate their ability to capture predictive signals from large-scale marker gene surveys of complex microbial communities.

In [ ]:
import pandas
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')
dataframe.head()

,DTA0,DTA1,DTA2,DTA3,DTA4,DTA5,DTA6,DTA7,DTA8,DTA9,...,DTA247,DTA248,DTA249,DTA250,DTA251,DTA252,DTA253,DTA254,DTA255,LBL0
0,1.92,1.80,1.44,1.79,1.68,1.42,1.52,1.58,1.43,1.45,...,-2.02,-2.32,-2.19,-2.25,-2.25,-2.29,-2.19,-2.63,-2.86,1.0
1,1.97,1.98,2.16,2.12,1.78,1.71,1.69,1.60,1.74,1.64,...,-2.05,-1.97,-1.92,-2.12,-1.94,-2.18,-2.45,-2.63,-2.87,0.0
2,2.25,2.11,2.05,1.92,2.08,1.93,1.87,1.57,1.81,1.61,...,-2.02,-1.87,-1.95,-2.09,-1.96,-1.99,-2.01,-2.57,-2.71,1.0
3,2.25,2.07,1.92,1.84,1.83,1.80,1.88,1.48,1.70,1.46,...,-1.94,-2.11,-2.22,-1.98,-2.22,-2.00,-2.10,-2.59,-2.84,0.0
4,2.28,2.27,2.26,2.20,2.01,2.00,1.99,1.92,1.68,1.79,...,-1.69,-1.66,-1.82,-1.88,-1.92,-1.89,-2.07,-2.50,-2.72,0.0


The dataset contains 256 columns labeled DTA0 through DTA255, representing the relative abundances of 256 bacterial taxa across the 16,344 samples. Relative abundance is calculated as the proportion of sequencing reads assigned to each taxon in a given sample. To account for sequencing depth differences, the relative abundances have been normalized using a center log-ratio transformation - a standard technique for compositional microbial data. This transforms the data to a log scale centered at zero, where positive values indicate higher than average abundance, and negative values indicate lower than average abundance for a given taxon. After transformation, the relative abundance values typically range from +2.5 to -2.5.

The target prediction variable is contained in the LBL0 column, with 0 indicating control samples and 1 indicating case samples positive for type 1 diabetes. Our modeling goal is to accurately predict this disease label based on the multivariate microbial community profiles in columns DTA0 through DTA255.

We first split the data into training and validation subsets for model fitting and evaluation. We then extract the DTA columns as model inputs and the LBL0 column as labels. To leverage TensorFlow's sequence modeling capabilities, we expand the data to three dimensions, with the additional dimension representing the ordered sequence of taxa.

In the following sections, we fit and compare several neural network architectures, including convolutional and recurrent networks, to evaluate their ability to capture predictive signals about disease status from the full set of microbial taxa abundances and their co-occurrence patterns.

In [ ]:
import numpy as np

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)
print(train_dataframe.shape, valid_dataframe.shape, dataframe.shape)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)
print(train_x.shape, valid_x.shape)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()
print(train_y.shape, valid_y.shape)

(13075, 257) (3269, 257) (16344, 257)
(13075, 256, 1) (3269, 256, 1)
(13075,) (3269,)


We can see that there are 16,344 total samples in our dataframe. We've extracted 13,057 for training and 3,269 for validation.

After expanding the data dimension, we have explanatory variables of shape (256,1), a one-dimensional sequence of 256 bacterial 'species'.

## simple linear model

Now that we have some training and validation data, we just need to build a classifier to predict disease risk from the data.

First, we'll start with a simple linear model implemented using a single Dense neuron with sigmoid output, as this is a binary classification problem.

In previous cases, we were able to send the data directly to the Dense layer. But in this case, because we have 'expanded' the last dimension of the data - in order to fit tensorflow's sequence models - we need to 'collapse' that dimension back down, so it can be properly analyzed by the Dense layer.

It's pretty easy to do this; we just need to use a Flatten layer to 'flatten' the 'expanded' data back down to a simple vector. And, because the Flatten layer is just like any other tensorflow Layer object, we can use it as the *first* layer in our network, provided we set the input_shape option.

The following code cell implements a simple linear classifier for our disease-risk prediction problem.

In [ ]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(256,1)))
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
409/409 [==============================] - 2s 4ms/step - loss: 0.6910 - accuracy: 0.5468 - val_loss: 0.6895 - val_accuracy: 0.5216
Epoch 2/20
409/409 [==============================] - 2s 6ms/step - loss: 0.6686 - accuracy: 0.6028 - val_loss: 0.6711 - val_accuracy: 0.5418
Epoch 3/20
409/409 [==============================] - 1s 3ms/step - loss: 0.6516 - accuracy: 0.6481 - val_loss: 0.6547 - val_accuracy: 0

Our initial baseline model is a simple linear classifier with only 257 trainable parameters - 256 input weights plus a single bias term. Despite its simplicity, it achieves solid performance, reaching approximately 80% training accuracy and 78% validation accuracy after 20 epochs. This indicates that a substantial amount of predictive signal for the disease labels is contained in a linear combination of the microbial taxa abundances. The linear model provides a reasonable baseline to benchmark more complex nonlinear models against. We now explore whether convolutional and recurrent neural network architectures can capture higher-order nonlinear relationships and covariation in the community structure to improve classification accuracy beyond this linear baseline.

## convolution model

In the next code section, we implement a basic one-layer convolutional neural network for disease classification. The convolution layer consists of 16 filters of length 3 applied with same padding, such that the output sequence length remains 256. We use a tanh activation, which is less common than ReLU for modern convnets but allows for more direct comparison to the default activation in recurrent networks. With 4,161 trainable weights, this nonlinear model has substantially more parameters than the linear baseline. To account for this increased model capacity, we train for 50 epochs which is longer than the linear model. This convolution network allows us to model localized nonlinear interactions among adjacent taxa in the community profiles. We can evaluate whether directly capturing these short-range taxon co-occurrence patterns improves predictive performance beyond the linear model.

In [ ]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=16,
                                 kernel_size=(3,),
                                 activation=tf.keras.activations.tanh,
                                 padding='same',
                                 input_shape=(256,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 256, 16)           64        
                                                                 
 flatten_2 (Flatten)         (None, 4096)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 4161 (16.25 KB)
Trainable params: 4161 (16.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
409/409 [==============================] - 1s 1ms/step - loss: 0.6752 - accuracy: 0.5778 - val_loss: 0.6363 - val_accuracy: 0.7167
Epoch 2/50
409/409 [==============================] - 0s 1ms/step - loss: 0.6129 - accuracy: 0.6747 - val_loss: 0.5996 - val_accuracy: 0.6455
Epoch 3/50
409/409 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7339 - val_loss: 0.5484 - val_accuracy: 0.7222
Epoch 4/50
409/409 [==============================] - 0s 1ms/step - loss: 0.5100 - accuracy: 0.7623 - val_loss: 0.5200 - val_accuracy: 0.7485
Epoch 5/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4852 - accuracy: 0.7798 - val_loss: 0.4999 - val_accuracy: 0.7589
Epoch 6/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4663 - accuracy: 0.7920 - val_loss: 0.4843 - val_accuracy: 0.7645
Epoch 7/50
409/409 [==============================] - 0s 1ms/step - loss: 0.4516 - accuracy: 0.7998 - val_loss: 0.4722 - val_accuracy: 0.7687
Epoch 

## LSTM model

In [ ]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=16, return_sequences=True, input_shape=(256,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()


# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256, 16)           1152      
                                                                 
 flatten_3 (Flatten)         (None, 4096)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 5249 (20.50 KB)
Trainable params: 5249 (20.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
409/409 [==============================] - 11s 25ms/step - loss: 0.6791 - accuracy: 0.5606 - val_loss: 0.6420 - val_accuracy: 0.7054
Epoch 2/50
409/409 [==============================] - 10s 25ms/step - loss: 0.6165 - accuracy: 0.6766 - val_loss: 0.6013 - val_accuracy: 0.6601
Epoch 3/50
409/409 [==============================] - 11s 27ms/step - loss: 0.5451 - accuracy: 0.7407 - val_loss: 0.5271 - val_accuracy: 0.7443
Epoch 4/50
409/409 [==============================] - 11s 26ms/step - loss: 0.4920 - accuracy: 0.7725 - val_loss: 0.5017 - val_accuracy: 0.7489
Epoch 5/50
409/409 [==============================] - 10s 26ms/step - loss: 0.4624 - accuracy: 0.7846 - val_loss: 0.4892 - val_accuracy: 0.7525
Epoch 6/50
409/409 [==============================] - 10s 25ms/step - loss: 0.4431 - accuracy: 0.7966 - val_loss: 0.4839 - val_accuracy: 0.7571
Epoch 7/50
409/409 [==============================] - 10s 25ms/step - loss: 0.4292 - accuracy: 0.8054 - val_loss: 0.4808 - val_accuracy:

# Basic Transformer Model

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, roc_auc_score

# download data
dataframe = pd.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model using functional api
repdim = 4 # set internal data representation dimensionality

# input and linear projection
inlayer = tf.keras.Input(shape=(256, 2))
proj = tf.keras.layers.Dense(units=repdim)(inlayer)

# multi-head attention block
mha1 = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=repdim)(proj, proj, proj)
res1 = tf.keras.layers.Add()([proj, mha1])
nrm1 = tf.keras.layers.LayerNormalization()(res1)

# feed-forward block
ffa1 = tf.keras.layers.Dense(units=repdim, activation=tf.keras.activations.relu)(nrm1)
ffb1 = tf.keras.layers.Dense(units=repdim)(ffa1)
res2 = tf.keras.layers.Add()([nrm1, ffb1])
nrm2 = tf.keras.layers.LayerNormalization()(res2)

# classification block
flt = tf.keras.layers.Flatten()(nrm2)
outlayer = tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(flt)

model = tf.keras.Model(inputs=inlayer, outputs=outlayer)
model.summary()

# compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)




Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 2)]             0         []                            
                                                                                                  
 dense_20 (Dense)            (None, 256, 4)               12        ['input_6[0][0]']             
                                                                                                  
 multi_head_attention_5 (Mu  (None, 256, 4)               156       ['dense_20[0][0]',            
 ltiHeadAttention)                                                   'dense_20[0][0]',            
                                                                     'dense_20[0][0]']            
                                                                                            

Epoch 1/50
409/409 [==============================] - 6s 13ms/step - loss: 0.7063 - accuracy: 0.5259 - precision: 0.5197 - recall: 0.5022 - auc: 0.5345 - val_loss: 0.7007 - val_accuracy: 0.5222 - val_precision: 0.5180 - val_recall: 0.9546 - val_auc: 0.5785
Epoch 2/50
409/409 [==============================] - 6s 14ms/step - loss: 0.6784 - accuracy: 0.5660 - precision: 0.5617 - recall: 0.5437 - auc: 0.5976 - val_loss: 0.6570 - val_accuracy: 0.5990 - val_precision: 0.5691 - val_recall: 0.8906 - val_auc: 0.6941
Epoch 3/50
409/409 [==============================] - 5s 13ms/step - loss: 0.6347 - accuracy: 0.6382 - precision: 0.6363 - recall: 0.6208 - auc: 0.6922 - val_loss: 0.6172 - val_accuracy: 0.6497 - val_precision: 0.7927 - val_recall: 0.4274 - val_auc: 0.7690
Epoch 4/50
409/409 [==============================] - 5s 13ms/step - loss: 0.5874 - accuracy: 0.6892 - precision: 0.6901 - recall: 0.6706 - auc: 0.7561 - val_loss: 0.6510 - val_accuracy: 0.6133 - val_precision: 0.8895 - val_recal

## Increased Complexity

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from tensorflow.keras import regularizers
tf.keras.optimizers.legacy.Adam

# download data
dataframe = pd.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# Build and compile the model
repdim = 32  # Internal data representation dimensionality
inlayer = tf.keras.Input(shape=(256, 2))
proj = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(inlayer)
mha1 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(proj, proj, proj)
res1 = tf.keras.layers.Add()([proj, mha1])
nrm1 = tf.keras.layers.LayerNormalization()(res1)
mha2 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(nrm1, nrm1, nrm1)
res2 = tf.keras.layers.Add()([nrm1, mha2])
nrm2 = tf.keras.layers.LayerNormalization()(res2)
ffa1 = tf.keras.layers.Dense(units=repdim*4, activation='relu')(nrm2)
ffb1 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa1)
res3 = tf.keras.layers.Add()([nrm2, ffb1])
nrm3 = tf.keras.layers.LayerNormalization()(res3)
ffa2 = tf.keras.layers.Dense(units=repdim*4, activation='relu')(nrm3)
ffb2 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa2)
res5 = tf.keras.layers.Add()([nrm3, ffb2])
nrm5 = tf.keras.layers.LayerNormalization()(res5)
dropout_layer = tf.keras.layers.Dropout(0.4)(nrm5)
flt = tf.keras.layers.Flatten()(dropout_layer)
outlayer = tf.keras.layers.Dense(units=1, activation='sigmoid')(flt)

model = tf.keras.Model(inputs=inlayer, outputs=outlayer)

# compile model
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(train_data, epochs=70, validation_data=valid_data, callbacks=[early_stopping])

model.summary()



Epoch 1/70
409/409 [==============================] - 26s 27ms/step - loss: 0.8464 - accuracy: 0.6025 - precision: 0.5976 - recall: 0.5927 - auc: 0.6362 - val_loss: 0.7359 - val_accuracy: 0.6152 - val_precision: 0.8967 - val_recall: 0.2803 - val_auc: 0.8429
Epoch 2/70
409/409 [==============================] - 11s 27ms/step - loss: 0.6126 - accuracy: 0.7276 - precision: 0.7251 - recall: 0.7206 - auc: 0.8040 - val_loss: 0.6336 - val_accuracy: 0.6947 - val_precision: 0.9061 - val_recall: 0.4501 - val_auc: 0.8758
Epoch 3/70
409/409 [==============================] - 10s 23ms/step - loss: 0.5410 - accuracy: 0.7635 - precision: 0.7612 - recall: 0.7581 - auc: 0.8469 - val_loss: 0.6368 - val_accuracy: 0.7008 - val_precision: 0.9202 - val_recall: 0.4549 - val_auc: 0.8815
Epoch 4/70
409/409 [==============================] - 10s 25ms/step - loss: 0.5100 - accuracy: 0.7759 - precision: 0.7735 - recall: 0.7711 - auc: 0.8615 - val_loss: 0.5196 - val_accuracy: 0.7611 - val_precision: 0.9145 - val_r

## Bi-Directional LSTM  + Transformer Block Integration

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, roc_auc_score
from tensorflow.keras import regularizers
tf.keras.optimizers.legacy.Adam


# download data
dataframe = pd.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)


# Build model using functional API
repdim = 16  # Set internal data representation dimensionality

inlayer = tf.keras.Input(shape=(256, 2))

# LSTM layer

# Bidirectional LSTM layer
lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=16, return_sequences=True))(inlayer)

# Dense projection layer
proj = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(lstm_layer)

# First multi-head attention block
mha1 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(proj, proj, proj)
res1 = tf.keras.layers.Add()([proj, mha1])
nrm1 = tf.keras.layers.LayerNormalization()(res1)

# Second multi-head attention block
mha2 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(nrm1, nrm1, nrm1)
res2 = tf.keras.layers.Add()([nrm1, mha2])
nrm2 = tf.keras.layers.LayerNormalization()(res2)

# Feed-forward block for the first two multi-heads
ffa1 = tf.keras.layers.Dense(units=repdim, activation='relu')(nrm2)
ffb1 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa1)
res3 = tf.keras.layers.Add()([nrm2, ffb1])
nrm3 = tf.keras.layers.LayerNormalization()(res3)

# Feed-forward block for the third multi-head
ffa2 = tf.keras.layers.Dense(units=repdim, activation='relu')(nrm3)
ffb2 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa2)
res5 = tf.keras.layers.Add()([nrm3, ffb2])
nrm5 = tf.keras.layers.LayerNormalization()(res5)

# Add dropout
dropout_layer = tf.keras.layers.Dropout(0.4)(nrm5)

# Flatten and classification block
flt = tf.keras.layers.Flatten()(dropout_layer)
outlayer = tf.keras.layers.Dense(units=1, activation='sigmoid')(flt)

# Create the model
model = tf.keras.Model(inputs=inlayer, outputs=outlayer)

# Build and compile the model
# compile model
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Fit model on training data
model.fit(
    train_data,
    epochs=70,
    validation_data=valid_data,
    callbacks=[early_stopping]
)

# Model summary
model.summary()


Epoch 1/70
409/409 [==============================] - 33s 45ms/step - loss: 0.7950 - accuracy: 0.5826 - precision: 0.5777 - recall: 0.5699 - auc: 0.6170 - val_loss: 0.7830 - val_accuracy: 0.6008 - val_precision: 0.9532 - val_recall: 0.2313 - val_auc: 0.8298
Epoch 2/70
409/409 [==============================] - 16s 39ms/step - loss: 0.5793 - accuracy: 0.7277 - precision: 0.7258 - recall: 0.7195 - auc: 0.8103 - val_loss: 0.5728 - val_accuracy: 0.7250 - val_precision: 0.9368 - val_recall: 0.4961 - val_auc: 0.8890
Epoch 3/70
409/409 [==============================] - 16s 38ms/step - loss: 0.5273 - accuracy: 0.7664 - precision: 0.7663 - recall: 0.7570 - auc: 0.8473 - val_loss: 0.4714 - val_accuracy: 0.8033 - val_precision: 0.7702 - val_recall: 0.8775 - val_auc: 0.8962
Epoch 4/70
409/409 [==============================] - 16s 38ms/step - loss: 0.4920 - accuracy: 0.7874 - precision: 0.7852 - recall: 0.7828 - auc: 0.8687 - val_loss: 0.5307 - val_accuracy: 0.7464 - val_precision: 0.6782 - val_r

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import roc_auc_score

# Load your data
dataframe = pd.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# Extract explanatory variables and label
dta_ids = [x for x in dataframe.columns if x.startswith('DTA')]
X = dataframe[dta_ids].to_numpy()
y = dataframe['LBL0'].to_numpy()

# Add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=X.shape[1])
X = np.stack([X, np.array([loc] * X.shape[0])], axis=-1)

# Train-validation split
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Package data into TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)
# Define the model building function
def build_model():
    repdim = 32  # Set internal data representation dimensionality
    inlayer = tf.keras.Input(shape=(256, 2))
    proj = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(inlayer)
    mha1 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(proj, proj, proj)
    res1 = tf.keras.layers.Add()([proj, mha1])
    nrm1 = tf.keras.layers.LayerNormalization()(res1)
    mha2 = tf.keras.layers.MultiHeadAttention(num_heads=14, key_dim=repdim)(nrm1, nrm1, nrm1)
    res2 = tf.keras.layers.Add()([nrm1, mha2])
    nrm2 = tf.keras.layers.LayerNormalization()(res2)
    ffa1 = tf.keras.layers.Dense(units=repdim*4, activation='relu')(nrm2)
    ffb1 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa1)
    res3 = tf.keras.layers.Add()([nrm2, ffb1])
    nrm3 = tf.keras.layers.LayerNormalization()(res3)
    ffa2 = tf.keras.layers.Dense(units=repdim*4, activation='relu')(nrm3)
    ffb2 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa2)
    res5 = tf.keras.layers.Add()([nrm3, ffb2])
    nrm5 = tf.keras.layers.LayerNormalization()(res5)
    dropout_layer = tf.keras.layers.Dropout(0.4)(nrm5)
    flt = tf.keras.layers.Flatten()(dropout_layer)
    outlayer = tf.keras.layers.Dense(units=1, activation='sigmoid')(flt)
    model = tf.keras.Model(inputs=inlayer, outputs=outlayer)
    return model

# Build and compile the model
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Fit model on training data
history = model.fit(
    train_data,
    epochs=70,
    validation_data=valid_data,
    callbacks=[early_stopping]
)

# Model summary
model.summary()


Epoch 1/70
409/409 [==============================] - 24s 51ms/step - loss: 0.9013 - accuracy: 0.5639 - precision_1: 0.5606 - recall_1: 0.5587 - val_loss: 0.6014 - val_accuracy: 0.7583 - val_precision_1: 0.7933 - val_recall_1: 0.6981
Epoch 2/70
409/409 [==============================] - 21s 51ms/step - loss: 0.6340 - accuracy: 0.7170 - precision_1: 0.7141 - recall_1: 0.7161 - val_loss: 0.5529 - val_accuracy: 0.7583 - val_precision_1: 0.8475 - val_recall_1: 0.6295
Epoch 3/70
409/409 [==============================] - 21s 51ms/step - loss: 0.5630 - accuracy: 0.7521 - precision_1: 0.7524 - recall_1: 0.7456 - val_loss: 0.4894 - val_accuracy: 0.8027 - val_precision_1: 0.7702 - val_recall_1: 0.8622
Epoch 4/70
409/409 [==============================] - 21s 51ms/step - loss: 0.5241 - accuracy: 0.7709 - precision_1: 0.7703 - recall_1: 0.7665 - val_loss: 0.4702 - val_accuracy: 0.8067 - val_precision_1: 0.7839 - val_recall_1: 0.8463
Epoch 5/70
409/409 [==============================] - 21s 50ms/s

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

# Load your data
dataframe = pd.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# Extract explanatory variables and label
dta_ids = [x for x in dataframe.columns if x.startswith('DTA')]
X = dataframe[dta_ids].to_numpy()
y = dataframe['LBL0'].to_numpy()

# Add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=X.shape[1])
X = np.stack([X, np.array([loc] * X.shape[0])], axis=-1)

# Train-validation split
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Package data into TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)


# Define the model building function
def build_model():

  # Build model using functional API
  repdim = 32  # Set internal data representation dimensionality

  inlayer = tf.keras.Input(shape=(256, 2))

  # LSTM layer

  # Bidirectional LSTM layer
  lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=12, return_sequences=True))(inlayer)

  # Dense projection layer (if needed to adjust dimensions)
  proj = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(lstm_layer)

  # First multi-head attention block
  mha1 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=repdim)(proj, proj, proj)
  res1 = tf.keras.layers.Add()([proj, mha1])
  nrm1 = tf.keras.layers.LayerNormalization()(res1)

  # Second multi-head attention block
  mha2 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=repdim)(nrm1, nrm1, nrm1)
  res2 = tf.keras.layers.Add()([nrm1, mha2])
  nrm2 = tf.keras.layers.LayerNormalization()(res2)

  # Feed-forward block for the first two multi-heads
  ffa1 = tf.keras.layers.Dense(units=repdim, activation='relu')(nrm2)
  ffb1 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa1)
  res3 = tf.keras.layers.Add()([nrm2, ffb1])
  nrm3 = tf.keras.layers.LayerNormalization()(res3)

  # Feed-forward block for the third multi-head
  ffa2 = tf.keras.layers.Dense(units=repdim, activation='relu')(nrm3)
  ffb2 = tf.keras.layers.Dense(units=repdim, kernel_regularizer=regularizers.l2(0.001))(ffa2)
  res5 = tf.keras.layers.Add()([nrm3, ffb2])
  nrm5 = tf.keras.layers.LayerNormalization()(res5)

  # Add dropout
  dropout_layer = tf.keras.layers.Dropout(0.4)(nrm5)

  # Flatten and classification block
  flt = tf.keras.layers.Flatten()(dropout_layer)
  outlayer = tf.keras.layers.Dense(units=1, activation='sigmoid')(flt)

  # Create the model
  model = tf.keras.Model(inputs=inlayer, outputs=outlayer)
  model = tf.keras.Model(inputs=inlayer, outputs=outlayer)
  return model
# Build and compile the model
model = build_model()
adam_optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Fit model on training data
history = model.fit(
    train_data,
    epochs=70,
    validation_data=valid_data,
    callbacks=[early_stopping]
)

# Predictions and evaluation on validation data
valid_predictions = model.predict(valid_x)
auroc_score = roc_auc_score(valid_y, valid_predictions)
print(f'Validation AUROC: {auroc_score}')

# Model summary
model.summary()


Epoch 1/70
409/409 [==============================] - 34s 53ms/step - loss: 0.8777 - accuracy: 0.5420 - precision: 0.5386 - recall: 0.5345 - val_loss: 0.6727 - val_accuracy: 0.7385 - val_precision: 0.7020 - val_recall: 0.8279
Epoch 2/70
409/409 [==============================] - 21s 50ms/step - loss: 0.7385 - accuracy: 0.6494 - precision: 0.6474 - recall: 0.6436 - val_loss: 0.5603 - val_accuracy: 0.7880 - val_precision: 0.7652 - val_recall: 0.8304
Epoch 3/70
409/409 [==============================] - 20s 49ms/step - loss: 0.6472 - accuracy: 0.7150 - precision: 0.7141 - recall: 0.7095 - val_loss: 0.5146 - val_accuracy: 0.8100 - val_precision: 0.7741 - val_recall: 0.8751
Epoch 4/70
409/409 [==============================] - 21s 52ms/step - loss: 0.6047 - accuracy: 0.7459 - precision: 0.7441 - recall: 0.7431 - val_loss: 0.4886 - val_accuracy: 0.8275 - val_precision: 0.7974 - val_recall: 0.8775
Epoch 5/70
409/409 [==============================] - 21s 51ms/step - loss: 0.5838 - accuracy: 0